In [17]:
from ortools.sat.python import cp_model

In [18]:
model = cp_model.CpModel()

x = model.new_int_var(0,2,"X")
y = model.new_int_var(0,2,"Y")
z = model.new_int_var(0,2,"Z")

model.add( x!=y)

solve =cp_model.CpSolver()
status = solve.Solve(model)

In [19]:
class PrintCSP(cp_model.CpSolverSolutionCallback):
    def __init__(self,var):
        super().__init__()
        self.variables = var
        self.solcnt = 0
    def on_solution_callback(self):
        self.solcnt += 1
        print(f"Solution: {self.solcnt} X: {self.Value(self.variables[0])} Y: {self.Value(self.variables[1])} Z: {self.Value(self.variables[2])}")


In [20]:
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print(f"X: {solve.value(x)}, Y: {solve.value(y)}, Z: {solve.value(z)}")

X: 1, Y: 0, Z: 0


In [21]:
prsol = PrintCSP([x,y,z])
status = solve.SearchForAllSolutions(model,prsol)

Solution: 1 X: 1 Y: 0 Z: 0
Solution: 2 X: 2 Y: 0 Z: 0
Solution: 3 X: 2 Y: 0 Z: 1
Solution: 4 X: 1 Y: 0 Z: 1
Solution: 5 X: 2 Y: 1 Z: 1
Solution: 6 X: 2 Y: 1 Z: 0
Solution: 7 X: 2 Y: 1 Z: 2
Solution: 8 X: 2 Y: 0 Z: 2
Solution: 9 X: 1 Y: 0 Z: 2
Solution: 10 X: 0 Y: 1 Z: 2
Solution: 11 X: 0 Y: 1 Z: 1
Solution: 12 X: 0 Y: 2 Z: 1
Solution: 13 X: 0 Y: 2 Z: 2
Solution: 14 X: 1 Y: 2 Z: 2
Solution: 15 X: 1 Y: 2 Z: 1
Solution: 16 X: 1 Y: 2 Z: 0
Solution: 17 X: 0 Y: 2 Z: 0
Solution: 18 X: 0 Y: 1 Z: 0


In [22]:

from ortools.sat.python import cp_model
model = cp_model.CpModel()
def times_overlap(time1, time2):
    return not (time1[1] <= time2[0] or time2[1] <= time1[0])
instructor = {'X':0, 'Y':1, 'Z':2}
ins = ["X","Y","Z"]
workshop ={
    1: (9.0,10.0),
    2: (9.5,10.5),
    3: (10.0,11.0),
    4: (10.5,11.5),
    5: (11.0,12.0)
}

domains= {
    1: [instructor['X'],instructor['Z']],
    2: [instructor['X'],instructor['Y']],
    3: [instructor['Y'],instructor['Z']],
    4: [instructor['Y'],instructor['Z']],
    5: [instructor['X']]
}

variables = {}

for x in range(1,6):
    variables[x] = model.NewIntVarFromDomain(cp_model.Domain.FromValues(domains[x]),f"workshop_{x}")

for w1 in range(1,6):
    for w2 in range(1,6):
        if w1<w2 and times_overlap(workshop[w1],workshop[w2]):
            model.add(variables[w1] != variables[w2])


In [23]:
class CSPPrinter(cp_model.CpSolverSolutionCallback):
    def __init__(self, variables, prof_names):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.variables = variables
        self.prof_names = prof_names
        self.solution_count = 0

    def on_solution_callback(self):
        self.solution_count += 1
        print(f'\nSolution {self.solution_count}:')
        for c in sorted(self.variables):
            assigned = self.Value(self.variables[c])
            print(f'Workshop{c} -> Instructor {self.prof_names[assigned]}')

In [24]:
solver = cp_model.CpSolver()
prsol = CSPPrinter(variables,ins)
status = solver.SearchForAllSolutions(model,prsol)


Solution 1:
Workshop1 -> Instructor Z
Workshop2 -> Instructor X
Workshop3 -> Instructor Y
Workshop4 -> Instructor Z
Workshop5 -> Instructor X

Solution 2:
Workshop1 -> Instructor Z
Workshop2 -> Instructor X
Workshop3 -> Instructor Z
Workshop4 -> Instructor Y
Workshop5 -> Instructor X

Solution 3:
Workshop1 -> Instructor Z
Workshop2 -> Instructor Y
Workshop3 -> Instructor Z
Workshop4 -> Instructor Y
Workshop5 -> Instructor X

Solution 4:
Workshop1 -> Instructor X
Workshop2 -> Instructor Y
Workshop3 -> Instructor Z
Workshop4 -> Instructor Y
Workshop5 -> Instructor X


In [25]:
solver = cp_model.CpSolver()
status = solver.solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for x in range(1,6):
        print(f"Workshop : {x} Assigned Instructor: {solver.value(variables[x])}")

Workshop : 1 Assigned Instructor: 2
Workshop : 2 Assigned Instructor: 0
Workshop : 3 Assigned Instructor: 1
Workshop : 4 Assigned Instructor: 2
Workshop : 5 Assigned Instructor: 0
